## Practica 1 - Computer Vision

Cargamos las librerías que vamos a emplear a lo largo de la práctica

In [26]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import os
import gdown

In [31]:
import os
import gdown

# Crear el directorio de datos si no existe
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# URL de Google Drive en formato correcto para gdown
url = "https://drive.google.com/uc?id=1iGBv-VT5mm1RiouD-U2qWcU3BYqp2OwE"
zip_filename = "practica_1_dataset.zip"
zip_path = os.path.join(data_dir, zip_filename)

# Descargar el archivo
gdown.download(url, zip_path, quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1iGBv-VT5mm1RiouD-U2qWcU3BYqp2OwE
From (redirected): https://drive.google.com/uc?id=1iGBv-VT5mm1RiouD-U2qWcU3BYqp2OwE&confirm=t&uuid=d4dba168-1597-4a5a-a066-e2b057a7fc8f
To: c:\Users\Rubén\Desktop\Universidad\cuarto_curso\dl\practica1\UFVDeepLearning\P1_ComputerVision\data\practica_1_dataset.zip
100%|██████████| 69.6M/69.6M [00:06<00:00, 11.5MB/s]


'data\\practica_1_dataset.zip'

In [32]:
import os
from zipfile import ZipFile
from concurrent.futures import ThreadPoolExecutor

data_dir = "data"
zip_filename = "practica_1_dataset.zip"
zip_path = os.path.join(data_dir, zip_filename)
if not os.path.exists(zip_path.split(".")[0]):
    with ZipFile(zip_path, 'r') as zf:
        with ThreadPoolExecutor() as exe:
            for file in zf.namelist():
                if not file.startswith("__MACOSX"):
                    exe.submit(zf.extract, file, path=data_dir)

In [ ]:
attributes = [
    "filename",
    "width",
    "height",
    "class",
    "xmin",
    "ymin",
    "xmax",
    "ymax"
]

PADDING

In [ ]:
max_height = None
max_width = None
